In [35]:
import os

from pandas.io.parquet import to_parquet

In [36]:
%pwd


'D:\\HourlyConsumptiomXGboost'

In [37]:
os.chdir(r"D:\HourlyConsumptiomXGboost")

In [38]:
%pwd

'D:\\HourlyConsumptiomXGboost'

In [39]:
# entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    # source_URL:str
    local_data_file: Path

In [40]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [41]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            # source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )

        return data_ingestion_config

In [42]:
import os
import urllib.request as request
import pandas as pd
from src.mlProject import logger
from src.mlProject.utils.common import get_size,get_mongoData


In [43]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
           
    def initiateDataIngestion(self):
        try:
            data = get_mongoData()
            logger.info("Reading Completed from MongoDB")
            logger.info("Writing data in Parquet file")
            data.to_parquet(self.config.local_data_file, index=False, engine='fastparquet')
            read_df = pd.read_parquet(self.config.local_data_file)
            print(read_df)
            
        except Exception as e:
            print(f"Error: {e}")
  

In [44]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.initiateDataIngestion()
    # data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-13 14:28:00,794: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-13 14:28:00,796: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-13 14:28:00,797: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-13 14:28:00,799: INFO: common: created directory at: artifacts]
[2023-10-13 14:28:00,800: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-13 14:28:00,802: INFO: common: calling DB configuration]
[2023-10-13 14:28:00,802: INFO: common: Reading data from Mongo DB]
[2023-10-13 14:28:31,028: INFO: common: Error occurs =========== 13.127.57.185:27017: timed out, Timeout: 30s, Topology Description: <TopologyDescription id: 65290698c336d6e8be458979, topology_type: Unknown, servers: [<ServerDescription ('13.127.57.185', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('13.127.57.185:27017: timed out')>]>]
[2023-10-13 14:28:31,030: INFO: 21609390: Reading Completed from MongoDB]
[2023-10-13 14:28: